In [ ]:
!pip install pypdf
!pip install -U langchain-community
!pip install sentence-transformers
!pip install -U bitsandbytes
!pip install PyPDF2
!pip install numpy==1.23.5
!pip install pandas
!pip install -U langchain-huggingface
!pip install accelerate>=0.26.0
!pip install fitz
!pip install frontend
!pip install pdfplumber
!pip install langchain PyPDFium2
!pip install faiss-gpu
!pip install grandalf
!pip install rank_bm25
!pip install tiktoken
!pip install -qU langchain-teddynote
!pip install matplotlib
!pip install chromadb
!pip install --upgrade langchain pydantic
!pip install pymupdf4llm
!pip install seaborn

In [ ]:
import os
import re
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from langchain.document_loaders import PyPDFium2Loader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from transformers import pipeline
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM
from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import pymupdf4llm
from langchain.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PDFMinerLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PDFMinerPDFasHTMLLoader
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_teddynote.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
import pdfplumber
import matplotlib.pyplot as plt

# Hyper Parameter

In [ ]:
RAG_Embedding_model = "jhgan/ko-sbert-nli"
# BAAI/bge-m3

TEST_Embedding = "jhgan/ko-sbert-sts"
LLM_Model ="CarrotAI/Llama-3.2-Rabbit-Ko-3B-Instruct-2412"
# "beomi/OPEN-SOLAR-KO-10.7B"
# "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"

model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}

# Text splitter
N_Chunk_size = 1000
N_Chunk_overlab = 100

R_k = 3

# Ensemble Retirever
# 1. CSV
# 2. PDF
Ensemble_rate = [0.5, 0.5] 

# Sample Test
Max_token_length = 100

# EmbeddingsFilter
# threshold = 0.8

# 1. Load PDF
## Preprocess PDF

In [ ]:
def clean_text(text):
    # 줄바꿈 제거
    #text = text.replace("\n", " ")  
    # 'KOSHA GUIDE' 제거
    text = text.replace("KOSHA GUIDE", "")  
    # 4자리 숫자(년도) 제거
    text = re.sub(r"\b\d{4}\b", "", text)
    # [참고'숫자'] 형태 제거
    text = re.sub(r"\[참고'\d+'\]", "", text)
    # '[글자]' 형태 제거
    text = re.sub(r"'\w+'", "", text)
    # '- 숫자 -' 형태 제거
    text = re.sub(r"-\s*\d+\s*-", "", text)
    # '<사진 숫자>' 형태 제거
    text = re.sub(r"<(사진|그림|표)\s*\d+>|(사진|그림|표)\s*\d+", "", text)
    # 불필요한 공백 정리
    text = re.sub(r"\s+", " ", text).strip()  
    return text

def load_pdfs_from_folder(folder_path):
    documents = []
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith(".pdf")]
    
    # PDF 파일 로드 진행 상황 표시
    for file_name in tqdm(pdf_files, desc="PDF 파일 로드 중", unit="파일"):
        file_path = os.path.join(folder_path, file_name)
        with pdfplumber.open(file_path) as pdf:
            # 각 페이지 로드 진행 상황 표시
            for i in tqdm(range(len(pdf.pages)), desc=f"{file_name} 페이지 로드 중", unit="페이지"):
                page = pdf.pages[i]
                # 표와 이미지를 제외하고 텍스트만 추출
                text = page.extract_text()
                if text:
                    # 전처리 적용
                    cleaned_text = clean_text(text)
                    if cleaned_text:  # 전처리 후에도 텍스트가 남아있다면 추가
                        documents.append({
                            "page_content": cleaned_text,
                            "metadata": {
                                "source": file_path,
                                "page": i + 1
                            }
                        })
    return documents

# PDF 문서 로드
pdf_docs = load_pdfs_from_folder("건설안전지침/")


In [ ]:
Page_num = 150
print(pdf_docs[Page_num])

# 2. Text Splitter

In [ ]:
def split_documents(documents, chunk_size=1000, chunk_overlap=100):
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", " ", ""],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        is_separator_regex=False,
    )

    chunks = []
    page_lengths = []

    for doc in documents:
        splits = text_splitter.split_text(doc["page_content"])

        page_length = len(doc["page_content"])
        page_lengths.append({
            "source": doc["metadata"]["source"],
            "page": doc["metadata"]["page"],
            "page_length": page_length
        })

        for split in splits:
            chunks.append({
                "source": doc["metadata"]["source"],
                "page_number": doc["metadata"]["page"],
                "text": split
            })

    return chunks, page_lengths

# PDF 문서에서 텍스트 분할 및 페이지 길이 확인
pdf_chunks, page_lengths = split_documents(pdf_docs)

In [ ]:
pdf_chunks[45]

In [ ]:
page_lengths_values = [p['page_length'] for p in page_lengths]

# 최대, 최소, 평균 길이 출력
max_length = max(page_lengths_values)
min_length = min(page_lengths_values)
avg_length = np.mean(page_lengths_values)

print(f"📏 최대 페이지 길이: {max_length}")
print(f"📏 최소 페이지 길이: {min_length}")
print(f"📏 평균 페이지 길이: {avg_length:.2f}")

# Reduce Page

In [ ]:
# page_lengths에서 인덱스와 page_length 추출
page_lengths_values_with_index = [(index, p['page_length']) for index, p in enumerate(page_lengths)]

# DataFrame 생성
df_page_lengths = pd.DataFrame(page_lengths_values_with_index, columns=["Index", "Page Length"])

# 결과 출력
df_page_lengths_filtered = df_page_lengths[df_page_lengths["Page Length"] <= 50]

# 결과 출력
df_page_lengths_filtered.head()

In [ ]:
# Define the bins with intervals of 100
bin_intervals = range(0, max_length + 100, 50)

# Plot the histogram with the new bin intervals
plt.figure(figsize=(10, 6))
plt.hist(page_lengths_values, bins=bin_intervals, color='skyblue', edgecolor='black')

# Add titles and labels
plt.title("Distribution of PDF Page Lengths")
plt.xlabel("Page Length (Number of Characters)")
plt.ylabel("Number of Pages")
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()

# Print the counts for each bin
counts, bin_edges = np.histogram(page_lengths_values, bins=bin_intervals)

# Display the bin ranges and corresponding counts
for i in range(len(counts)):
    print(f"Bin {bin_edges[i]} - {bin_edges[i+1]}: {counts[i]} pages")

In [ ]:
pdf_chunks[808]

In [ ]:
import seaborn as sns
# 'text'의 길이를 계산
text_lengths = [len(chunk['text']) for chunk in pdf_chunks]

# 분포도 그리기
plt.figure(figsize=(10, 6))
sns.histplot(text_lengths, bins=30, kde=True)
plt.title('PDF Chunks Text Length Distribution')
plt.xlabel('Text Length')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
# 0에서 1000 사이의 길이를 가진 요소의 수 확인
filtered_lengths = [length for length in text_lengths if 0 <= length <= 1000]

# 구간 설정
bin_intervals = range(0, 1050, 50)

# 각 구간의 개수 계산
counts, bin_edges = np.histogram(filtered_lengths, bins=bin_intervals)

# 각 구간의 범위와 개수 출력
for i in range(len(counts)):
    print(f"Bin {bin_edges[i]} - {bin_edges[i+1]}: {counts[i]} pages")

In [ ]:
# Bin 0 - 100에 해당하는 인덱스 찾기
bin_0_to_100_indices = [i for i, length in enumerate(filtered_lengths) if 0 <= length < 100]

# Bin 0 - 100에 해당하는 요소 제거
filtered_lengths = [length for i, length in enumerate(filtered_lengths) if i not in bin_0_to_100_indices]

In [ ]:
# 제거 후 결과 출력
print("제거 후 filtered_lengths의 길이:", len(filtered_lengths))

# ★ RAG (Retrieval Augmented Generation)

# 2. CSV Load

In [ ]:
# 5. 데이터 전처리 (Train & Test 데이터)
train = pd.read_csv('./train.csv', encoding='utf-8-sig')
test = pd.read_csv('./test.csv', encoding='utf-8-sig')

## 2-1 Column Drop

In [ ]:
# 특정 컬럼 제거
columns_to_drop = ['발생일시', '사고인지 시간', '날씨', '기온', '습도', '연면적', '층 정보']

# train 데이터에서 컬럼 제거
train = train.drop(columns=columns_to_drop)

# test 데이터에서 컬럼 제거
test = test.drop(columns=columns_to_drop)

In [ ]:
# 결측치 처리 (빈 문자열로 채우기)
train['공사종류'] = train['공사종류'].fillna('')
train['공종'] = train['공종'].fillna('')
train['사고객체'] = train['사고객체'].fillna('')
train['작업프로세스'] = train['작업프로세스'].fillna('')
train['사고원인'] = train['사고원인'].fillna('')

test['공사종류'] = test['공사종류'].fillna('')
test['공종'] = test['공종'].fillna('')
test['사고객체'] = test['사고객체'].fillna('')

In [ ]:
# 데이터 전처리
train['공사종류(대분류)'] = train['공사종류'].str.split(' / ').str.get(0)
train['공사종류(중분류)'] = train['공사종류'].str.split(' / ').str.get(1)

train['공종(대분류)'] = train['공종'].str.split(' > ').str[0]
train['공종(중분류)'] = train['공종'].str.split(' > ').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split(' > ').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split(' > ').str[1]

test['공사종류(대분류)'] = test['공사종류'].str.split(' / ').str[0]
test['공사종류(중분류)'] = test['공사종류'].str.split(' / ').str[1]
test['공종(대분류)'] = test['공종'].str.split(' > ').str[0]
test['공종(중분류)'] = test['공종'].str.split(' > ').str[1]
test['사고객체(대분류)'] = test['사고객체'].str.split(' > ').str[0]
test['사고객체(중분류)'] = test['사고객체'].str.split(' > ').str[1]

In [ ]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"'{row['작업프로세스']}', '{row['인적사고']}', '{row['사고원인']}' "
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

combined_training_data = pd.DataFrame(list(combined_training_data))

In [ ]:
combined_training_data = pd.concat([train['ID'], combined_training_data], axis=1)
combined_training_data['Type'] = 'csv'
combined_training_data.head(5)

In [ ]:
train_documents = [
    f"Q: {q1}\nA: {a1}" 
    for q1, a1, in zip(combined_training_data['question'], combined_training_data['answer'])
]

In [ ]:
train_documents[:5]

In [ ]:
import matplotlib.pyplot as plt

# train_documents의 각 요소의 길이 계산
lengths = [len(doc) for doc in train_documents]

# 최대, 최소, 평균 계산
max_length = np.max(lengths)
min_length = np.min(lengths)
mean_length = np.mean(lengths)

# 결과 출력
print(f"최대 길이: {max_length}")
print(f"최소 길이: {min_length}")
print(f"평균 길이: {mean_length:.2f}")

# 히스토그램 그리기
plt.figure(figsize=(10, 6))
plt.hist(lengths, bins=30, color='blue', alpha=0.7)
plt.title('Length of train_documents')
plt.xlabel('Length of Document')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.show()


# 3. Text Embedding

In [ ]:

# 6. 임베딩 모델 선택 (훈련 데이터)
embedding_model_name = RAG_Embedding_model
embedding = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# 4. Vector DB
## PDF -> FAISS
## CSV -> Chroma

In [ ]:
# 텍스트 추출
text_set = [chunk["text"] for chunk in pdf_chunks] 

# 메타데이터 추출 (Type 추가)
metadata_set = [
    {
        "source": chunk["source"], 
        "page_number": chunk["page_number"],
    } 
    for chunk in pdf_chunks
]

In [ ]:
# 메타데이터 추출
metadata_set_CSV = [
    {
        "ID": row["ID"],
        "공사종류(대분류)": row["공사종류(대분류)"],
        "공사종류(중분류)": row["공사종류(중분류)"],
        "공종(대분류)": row["공종(대분류)"],
        "공종(중분류)": row["공종(중분류)"],
        "사고객체(대분류)": row["사고객체(대분류)"],
        "사고객체(중분류)": row["사고객체(중분류)"]
    }
    for _, row in train.iterrows()
]


In [ ]:
# metadata_set_CSV를 DataFrame으로 변환
df_metadata = pd.DataFrame(metadata_set_CSV)

# NaN 값이 있는지 확인
nan_check = df_metadata.isna().any()

# 결과 출력
print("각 컬럼의 NaN 여부:\n", nan_check)

# NaN 값이 있는 행 추출
nan_rows = df_metadata[df_metadata.isna().any(axis=1)]

# NaN 값이 있는 행이 있다면 출력
if not nan_rows.empty:
    print("NaN 값을 포함한 행:\n", nan_rows)
else:
    print("NaN 값이 포함된 행이 없습니다.")


In [ ]:
metadata_set[:10]

In [ ]:
# Train
vectorstore = Chroma.from_texts(train_documents, embedding, metadatas=metadata_set_CSV)  

# PDF
vectorstore2 = FAISS.from_texts(text_set, embedding, metadatas=metadata_set) 

# 5. Retriever (Ensemble Retriever)

In [ ]:
# Train
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": R_k})

# PDF
retriever2 = vectorstore2.as_retriever(search_type="similarity", search_kwargs={"k": R_k})

In [ ]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, retriever2],
    weights=Ensemble_rate,
)

In [ ]:
# LLM 모델 로드 (4-bit 양자화)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(LLM_Model, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(LLM_Model)

In [ ]:
text_generation_pipeline = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    do_sample=False,  # 결정론적 생성
    return_full_text=False, 
    max_new_tokens=Max_token_length,
    device_map="auto"
)

# 경고를 비활성화할 수도 있습니다.
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='transformers')


# 6. Prompt Design

In [ ]:
prompt_template = """
Your mission is to be a safety expert answer bot. When given the {question}, you must provide an answer **only** if the question is related to **construction, safety, or safety measures**. 

If the question is **NOT** related to **construction, construction safety tasks, or construction safety plan tasks**, respond with:

### 답변 : "업무와 관련이 없는 질문입니다."
### 끝.

Your answer **MUST** be in **only one sentence.
Your answer **MUST** be written **only in Korean**.

If any of the above instructions are violated, a penalty will be applied.

Make sure to strictly follow these rules.

### 질문:
{question}
"""

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 커스텀 프롬프트 생성
prompt = PromptTemplate(
    input_variables=["question"],
    template=prompt_template,
)


# 7. Reference Check

In [ ]:
import random

Sample_num = 5

# combined_training_data에서 랜덤하게 5개 샘플 선택
sample_questions = random.sample(list(combined_training_data['question']), Sample_num)

# 각 샘플에 대해 compression_retriever에서 검색된 문서 확인
for idx, question in enumerate(sample_questions):
    print(f"샘플 {idx+1}: {question}\n")

    # 검색 수행
    retrieved_docs = ensemble_retriever.get_relevant_documents(question)

    print("검색된 문서들:")
    for doc_idx, doc in enumerate(retrieved_docs):
        print(f"Document {doc_idx+1}: {doc.page_content}")
        print(f"Meta Data : {doc.metadata}\n")  # 메타데이터 출력
    
    print("=" * 100)  # 가독성을 위한 구분선


# 8. RAG Chain

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
chain = (
    {"context": ensemble_retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# chain = (
#     {"context": ensemble_retriever|format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

# 9. Visualization in Langchain

In [ ]:
chain.get_graph().print_ascii()

# 10. Sample Test

In [ ]:
# 테스트 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"긴급상황입니다. 작업 프로세스는 '{row['작업프로세스']}'이며, 인적사고는 '{row['인적사고']}' 그리고 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        )
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))


In [ ]:
# 테스트 실행 후 결과 저장
test_results_t = []
metadata_list_t = []  # 메타데이터를 저장할 리스트

preview_count = 10

# 미리 보기 샘플 수
preview_data = combined_test_data.sample(n=preview_count)

print("테스트 실행 시작... 총 테스트 샘플 수:", preview_count)

# tqdm을 사용하여 진행 상황 표시
for idx, row in tqdm(preview_data.iterrows(), total=len(preview_data), desc="테스트 진행 중", unit="샘플"):
    # RAG 체인 호출 및 결과 생성
    prevention_result = chain.invoke(row['question'])

    if isinstance(prevention_result, str):
        result_text = prevention_result  # 문자열이라면 그 자체를 사용
    else:
        result_text = prevention_result.get('result', '')

    # 질문과 결과를 함께 저장
    test_results_t.append({"question": row['question'], "result": result_text})

    # 가장 높은 순위의 문서 메타데이터 수집
    retrieved_docs = ensemble_retriever.get_relevant_documents(row['question'])
    
    if retrieved_docs:
        top_doc = retrieved_docs[0]  # 가장 높은 순위의 문서

        if "source" in top_doc.metadata:  # PDF 문서일 경우
            metadata_list_t.append({
                "Type": 'pdf',
                "ID": None,
                "Title": top_doc.metadata["source"],
                "Page_number": top_doc.metadata["page_number"]
            })
        elif "ID" in top_doc.metadata:  # CSV 문서일 경우
            metadata_list_t.append({
                "Type": 'csv',
                "ID": top_doc.metadata["ID"],
                "Title": None,
                "Page_number": None
            })

print("\n테스트 실행 완료! 총 결과 수:", len(test_results_t))



In [ ]:

# 데이터프레임 생성
metadata_test = pd.DataFrame(metadata_list_t)
metadata_test

In [ ]:
# 각 질문에 대해 검색된 문서 및 답변 출력
for idx, result in enumerate(test_results_t):
    question = result["question"]
    answer = result["result"]  # 답변 가져오기
    print(f"질문 {idx + 1}: {question}\n")
    print(f"답변 {idx + 1}: {answer}\n")  # 답변 출력

    # 검색 수행
    retrieved_docs = ensemble_retriever.get_relevant_documents(question)

    print("검색된 문서들:")
    for doc_idx, doc in enumerate(retrieved_docs):
        print(f"Document {doc_idx + 1}: {doc.page_content}")
        print(f"Meta Data: {doc.metadata}\n")  # 메타데이터 출력

    print("=" * 100)  # 가독성을 위한 구분선


In [ ]:
# test_results_t를 데이터프레임으로 변환
results_df = pd.DataFrame(test_results_t)

# 두 데이터프레임을 합치기
combined_df = pd.concat([results_df, metadata_test], axis=1)
combined_df['result']

In [ ]:
combined_df['result'][0]

# 11. PDF based Question

In [ ]:
# 질문 리스트
questions = [
    "크레인 운전원이 지켜야할 안전 수칙에 대해 말하시오.",
    "곤돌라 작업대가 허용 하강 속도를 초과하면 어떤 조치를 취해야 하는가?",
    "암반 변화 구간의 발파 작업에는 무엇을 해야 하는가?",
    "부재의 현장 반입 시 해야되는 작업에 대해서 말하시오.",
    "단위공종별 작업 시작 전 해야되는 일들에 대해 설명하시오.",
    "시스템 동바리 조립시 작업자가 해야되는 업무에 대해 설명하시오.",
    "U자 걸이로 사용해야되는 안전대 '호' 번호는?",
    "야간 근로자의 작업 시간에 따른 휴식 시간은?",
    "내장 공사에서 작업 발판 사용시 폭의 기준은?",
    "캐트워크가 바람에 흔들리는 경우에 해야되는 조치는?"
]

# DataFrame 생성
PDF_question = pd.DataFrame({'question': questions})
# 테스트 실행 후 결과 저장
PDF_question['Result'] = ''  # 결과 저장 컬럼 추가
PDF_question['Type'] = ''  # 문서 유형

PDF_question['Doc_ID'] = None  # 문서 ID (CSV)
PDF_question['Title'] = ''  # 문서 제목 (PDF)
PDF_question['Page_number'] = None  # 페이지 번호 (PDF)

In [ ]:
# tqdm을 사용하여 진행 상황 표시
for idx, row in tqdm(PDF_question.iterrows(), total=len(PDF_question), desc="테스트 진행 중", unit="샘플"):
    # RAG 체인 호출 및 결과 생성
    prevention_result = chain.invoke(row['question'])
    
    if isinstance(prevention_result, str):
        PDF_question.at[idx, 'Result'] = prevention_result  # 문자열 결과 저장
    else:
        PDF_question.at[idx, 'Result'] = prevention_result.get('result', '')

    # 가장 높은 순위의 문서 메타데이터 수집
    retrieved_docs = ensemble_retriever.get_relevant_documents(row['question'])
    
    if retrieved_docs:
        top_doc = retrieved_docs[0]  # 가장 높은 순위의 문서
        metadata = top_doc.metadata

        if "source" in metadata:  # PDF 문서일 경우
            PDF_question.at[idx, 'Type'] = 'pdf'
            PDF_question.at[idx, 'Title'] = metadata["source"]
            PDF_question.at[idx, 'Page_number'] = metadata.get("page_number", None)
        elif "ID" in metadata:  # CSV 문서일 경우
            PDF_question.at[idx, 'Type'] = 'csv'
            PDF_question.at[idx, 'Doc_ID'] = metadata["ID"]

print("\n테스트 실행 완료! 총 결과 수:", len(PDF_question))


In [ ]:
PDF_question['Result']

# 12. Unrelated Question

In [ ]:
import pandas as pd
from tqdm import tqdm

# 엉뚱한 질문 리스트
unrelated_questions = [
    "하늘은 무슨 색인가요?",
    "한국에서 가장 유명한 음식은 무엇인가요?",
    "우주에서는 소리가 들릴까요?",
    "고양이는 왜 박스를 좋아하나요?",
    "로마 제국이 멸망한 이유는 무엇인가요?",
    "인간의 평균 수면 시간은 몇 시간인가요?",
    "이탈리아의 수도는?",
    "한국에서 가장 높은 산은 어디인가요?",
]

In [ ]:
# DataFrame 생성
Unrelated_questions = pd.DataFrame({'question': unrelated_questions})

# 테스트 실행 후 결과 저장
Unrelated_questions['Result'] = ''  # 결과 저장 컬럼 추가

In [ ]:
# tqdm을 사용하여 진행 상황 표시
for idx, row in tqdm(Unrelated_questions.iterrows(), total=len(Unrelated_questions), desc="테스트 진행 중", unit="샘플"):
    # RAG 체인 호출 및 결과 생성
    prevention_result = chain.invoke(row['question'])

    # prevention_result에 "업무와 관련이 없는 내용입니다."가 있으면 그 문장을 그대로 사용
    if isinstance(prevention_result, str):
        if "업무와 관련이" in prevention_result:
            Unrelated_questions.at[idx, 'Result'] = "업무와 관련이 없는 질문입니다."  # 해당 문장으로 변경
        else:
            Unrelated_questions.at[idx, 'Result'] = prevention_result  # 문자열 결과 저장
    else:
        result_text = prevention_result.get('result', '')
        if "업무와 관련이" in result_text:
            Unrelated_questions.at[idx, 'Result'] = "업무와 관련이 없는 질문입니다."  # 해당 문장으로 변경
        else:
            Unrelated_questions.at[idx, 'Result'] = result_text  # 결과 저장

    # 가장 높은 순위의 문서 메타데이터 수집
    retrieved_docs = ensemble_retriever.get_relevant_documents(row['question'])

    if retrieved_docs:
        top_doc = retrieved_docs[0]  # 가장 높은 순위의 문서

print("\n테스트 실행 완료! 총 결과 수:", len(Unrelated_questions))


In [ ]:
print(Unrelated_questions['Result'])

# 13. Inference & TEST

In [ ]:
import re
from tqdm import tqdm

# 테스트 실행 및 결과 저장
test_results = []
metadata_list = []

print("테스트 실행 시작... 총 테스트 샘플 수:", len(combined_test_data))

# tqdm을 사용하여 진행 상황 표시
for idx, row in tqdm(combined_test_data.iterrows(), total=len(combined_test_data), desc="테스트 진행 중", unit="샘플"):
    # RAG 체인 호출 및 결과 생성
    prevention_result = chain.invoke(row['question'])

    if isinstance(prevention_result, str):
        result_text = prevention_result
    else:
        result_text = prevention_result.get('result', '')

    # '답변' 이전 제거
    first_sentence = re.sub(r'.*답변\s*[:：]?\s*', '', result_text)
    
    # '답변 :' 또는 '답변:' 제거
    first_sentence = re.sub(r'답변\s*[:：]?\s*', '', first_sentence)

    # '.'와 ','을 제외한 특수문자 및 줄바꿈 문자 제거
    first_sentence = re.sub(r'[^가-힣0-9\s.,]', '', first_sentence).replace('\n', '')
    
    # 첫 번째 문장 끝나면 그 뒤의 내용 제거
    first_sentence = re.split(r'[.!?]\s', first_sentence, maxsplit=1)[0].strip()


    if "업무와 관련이" in first_sentence:
        first_sentence = "업무와 관련이 없는 질문입니다."

    # 결과 저장
    test_results.append({
        "question": row['question'],
        "result": first_sentence
    })

    # 가장 높은 순위의 문서 메타데이터 수집
    retrieved_docs = ensemble_retriever.get_relevant_documents(row['question'])
    
    if retrieved_docs:
        top_doc = retrieved_docs[0]  # 가장 높은 순위의 문서
        if "source" in top_doc.metadata:  # PDF 문서일 경우
            metadata_list.append({
                "Type": 'pdf',
                "ID": None,
                "Title": top_doc.metadata.get("source"),
                "Page_number": top_doc.metadata.get("page_number")
            })
        elif "ID" in top_doc.metadata:  # CSV 문서일 경우
            metadata_list.append({
                "Type": 'csv',
                "ID": top_doc.metadata.get("ID"),
                "Title": None,
                "Page_number": None
            })

print("\n테스트 실행 완료! 총 결과 수:", len(test_results))


In [ ]:
embedding_model_name = TEST_Embedding
embedding = SentenceTransformer(embedding_model_name)

In [ ]:
test_results[0]

In [ ]:
# 문장 리스트를 입력하여 임베딩 생성
# 만약 test_results가 리스트라면
if isinstance(test_results, list):
    pred_embeddings = [embedding.encode(result['result']) for result in test_results]
else:
    pred_embeddings = embedding.encode(test_results['result'])

In [ ]:
pred_embeddings[0]

In [ ]:
test_results[:10]

# 14. Submission

In [ ]:
# CSV 파일 읽기
submission = pd.read_csv('./sample_submission.csv', encoding='utf-8-sig')
# submission을 복사하여 submission2 생성
submission2 = submission.copy()

# 최종 결과 저장
# test_results가 리스트인 경우
if isinstance(test_results, list):
    results = [result['result'] for result in test_results]  # 각 result를 리스트로 변환
else:
    results = test_results['result']  # 단일 결과인 경우

# 결과를 submission에 할당
submission.iloc[:, 1] = results  # results를 submission의 두 번째 열에 할당

# pred_embeddings가 2차원 배열인 경우
submission.iloc[:, 2:] = pred_embeddings


In [ ]:
submission

In [ ]:
# 최종 결과를 CSV로 저장
submission.to_csv('./RAG_Ensemble_final_5.csv', index=False, encoding='utf-8-sig')

# 15. Add File for Qualitative Evaluation
### 1) Train : ID
### 2) PDF : Source(PDF Title), Page Number

In [ ]:
# 데이터프레임 생성
metadata = pd.DataFrame(metadata_list)
metadata

In [ ]:
# test_results_t를 데이터프레임으로 변환
results_set = pd.DataFrame(test_results)

# 두 데이터프레임을 합치기
Q_Evaluation = pd.concat([results_set, metadata], axis=1)
Q_Evaluation

In [ ]:
# 평가표
Q_Evaluation.to_csv('./Evaluation_metric_final5.csv', index=False, encoding='utf-8-sig')

In [ ]:
Q_Evaluation

In [ ]:
# "Type" 컬럼의 고유한 요소 출력
unique_types = combined_df['Type'].unique()

print("Type 컬럼의 고유한 요소:")
for t in unique_types:
    print(t)
